In [6]:
import os
# When using on Juypter Notebook, uncomment the code below
# import sys
# path = os.path.join(os.path.dirname(os.getcwd()))
# sys.path.append(path) 
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from utils.common_utils import printlog, make_dir
from utils.processing_tools.processing import *
from utils.feature_extraction_tools.feature_extraction import emg_feature_extraction
from utils.common_params import *

In [2]:
### Parameter Settings
## 1. Original file path
raw_file_path = 'SIAT_LLMD_rawData'

## 2. Subject and Exercise type Settings
subjects_list_global = list(['01','02','03','04','05', '31','32','33','34','35'])
motions_list_global = ['WAK', 'STDUP', 'SITDN', 'UPS', 'DNS'] 

## 3. Channel (column) name of emg in the raw data used
emg_name = ['sEMG: tensor fascia lata', 'sEMG: rectus femoris', 'sEMG: vastus medialis',
            'sEMG: semimembranosus', 'sEMG: upper tibialis anterior', 'sEMG: lower tibialis anterior',
            'sEMG: lateral gastrocnemius', 'sEMG: medial gastrocnemius', 'sEMG: soleus']

## 4. Data Preprocessing Settings
# 4.1. Downsampling: 1920 Hz - 1000 Hz
# 4.2. emg notch filter (3rd order FIR notch filter, 50Hz, band width 2Hz)
# 4.3. emg Butterworth filter (low cut-off frequency :15hz; High cut-off frequency :450hz; Order :7)
# 4.4. emg denoising
## 4.4.1 ICEEMDAN-PE-GT/SVD
## 4.4.1.1 ICEEMDAN: Noise amplitude, ε0 = 0.2; Number of the noise realizations, I=100;
## 4.4.1.2 Adaptive PE decision threshold
## 4.4.1.3GT: Threshold: improvement of the 'universal' threshold (visushrink); Threshold rule: garrote threshold
## 4.4.2 WD-GT(Wavelet denoising, wavelet basis' db5 ', decomposition level 3, Threshold rule: garrote threshold, Threshold universal threshold 'universal')
## 4.4.3 WPD-GT(wavelet packet denoising, wavelet basis' db5 ', decomposition level 3, Threshold rule: garrote threshold, threshold universal threshold 'universal')
# 4.5. emg normalization
emg_denoise_method = 'rawdata'  # ['rawdata', 'WD-GT', 'WPD-GT', 'EMD-PE-GT', 'EMD-PE-SVD', 'ICEEMDAN-PE-GT', 'ICEEMDAN-PE-SVD']
data_normalize_method = 'positive_negative_one' # 'min-max' (0, 1), 'max-abs' (-1,1), 'positive_negative_one' (-1, 1)
data_normalize_level = 'matrix' # 'matrix' / 'rows' 

## 5. Overlapping window sampling: overlapping window length, step length

## 6. Feature extraction settings for emg and joint angle data
emg_channels = ['TFL', 'RF', 'VM', 'SEM', 'UTA', 'LTA', 'LG', 'MG', 'SOL']
emg_feature_type = ['MAV', 'RMS', 'WL', 'ZC', 'SSC', 'WAMP']
fea_normalize_method = 'min-max'
fea_normalize_level = 'rows'

In [3]:
for i in range(len(subjects_list_global)):
    subject = 'Sub' + subjects_list_global[i]
    data_file_dir = os.path.join(raw_file_path,subject,'Data')
    label_file_dir = os.path.join(raw_file_path,subject,'Labels')
    sub_emg_sample, sub_emg_features, sub_motion_label_raw = [], [], []
    printlog(info = ''.join(['Start processing:  ', subject]), time = True, line_break = False)
    
    for j in range(len(motions_list_global)):
        motion = motions_list_global[j]
        printlog(info = "\n"+"********"*4 +''.join([' motion: ', motion, ' '])+"********"*4, time = False, line_break = False)
        printlog (info = 'step1: Load data and label...', time = False, line_break = False)
        ## 1.1 data and label filenames
        data_file_name = os.path.join(data_file_dir,''.join([subject, '_', motion, '_Data.csv']))
        label_file_name = os.path.join(label_file_dir,''.join([subject, '_', motion, '_Label.csv']))
        ## 1.2 read data and label
        df_data = pd.read_csv(data_file_name,header=0)
        time = df_data.loc[:, ['Time']].values
        emg_raw_data = df_data.loc[:, emg_name].values
        df_label = pd.read_csv(label_file_name,header=0)
        status_label = df_label.loc[:, ['Status']].values
        group_label = df_label.loc[:, ['Group']].values
        
        printlog ('step2: Data preprocessing...', time = False, line_break = False)
        ## 2.0 Determine whether an active segment signal is needed
        emg_act_data = get_emg_act_signal(motion, emg_raw_data, status_label)
        ## 2.1 Resampling
        emg_resample_data = signal2d_resampling(emg_act_data, raw_fs, tar_fs)
        ## 2.2 Filtering
        emg_filtered_data = emg_filtering(emg_resample_data, tar_fs)
        ## 2.3 Denosing
        emg_denoise_data = Signal2dDenoise(data=emg_filtered_data, denoise_method=emg_denoise_method).forward()
        ## 2.4 Normalization
        emg_data_pre = data_nomalize(emg_denoise_data, data_normalize_method, data_normalize_level)
        printlog ('step3: Overlapping window segmentation...', time = False, line_break = False)
        emg_sample, motion_label_raw = movement_classification_sample_segmentation(motion, emg_data_pre, window, step)
        printlog ('step4: Feature extraction...', time = False, line_break = False)
        emg_features, emg_feature_list = emg_feature_extraction(emg_sample, emg_channels, emg_feature_type, fea_normalize_method, fea_normalize_level)
        printlog ('step5: Merging: all emg data, features, labels...', time = False, line_break = False)
        sub_emg_sample.extend(emg_sample)
        sub_emg_features.extend(emg_features)
        sub_motion_label_raw.extend(motion_label_raw)
        
    printlog(info = ''.join(['Merging: ', subject, 'data for all motion types...']), time = True, line_break = False)
    sub_emg_sample = np.swapaxes(np.array(sub_emg_sample), 1, 2)   # Swap the second and third dimensions to become: num*len(emg_channels)*len(window)
    sub_emg_features, sub_motion_label_raw = np.array(sub_emg_features), np.array(sub_motion_label_raw)
    printlog(info = ''.join(['Subject: ', subject, 'numeric labels and one-hot labels for all motion types...']), time = False, line_break = False)
    label_encoder = LabelEncoder()
    sub_motion_label_encoded_raw = label_encoder.fit_transform(sub_motion_label_raw) 
    custom_order = motions_list_global
    label_encoder.classes_ = custom_order
    sub_motion_label_encoded = np.array([custom_order.index(label) for label in sub_motion_label_raw])
    onehot_encoder = OneHotEncoder(sparse_output=False) 
    sub_motion_label_onehot = onehot_encoder.fit_transform(sub_motion_label_encoded.reshape(-1, 1))
    printlog(info = ''.join(['Subject: ', subject, ', ', 'done!']), time = False, line_break = False)
    print('emg_sample.shape: ', sub_emg_sample.shape)
    print('emg_features.shape: ', sub_emg_features.shape)
    print('motion_label_raw.shape: ', sub_motion_label_raw.shape, ', motion_label_encoded.shape: ', sub_motion_label_encoded.shape, ', motion_label_onehot.shape: ', sub_motion_label_onehot.shape)
    
    file_save_path = os.path.join(os.getcwd(), 'SIAT_LLMD_trainData', emg_denoise_method)
    make_dir(file_save_path)
    file_save_name = os.path.join(file_save_path,''.join([subject,'_targetTrainData.npz']))
    with open(file_save_name, 'wb') as f:
        np.savez(f, sub_emg_sample = sub_emg_sample, sub_emg_features = sub_emg_features, sub_motion_label_raw = sub_motion_label_raw,
        sub_motion_label_encoded = sub_motion_label_encoded, sub_motion_label_onehot = sub_motion_label_onehot)
    printlog(info = file_save_name, time = False, line_break = False)


================================================================================2025-02-16 23:04:43
Start processing:  Sub01

******************************** motion: WAK ********************************
step1: Load data and label...
step2: Data preprocessing...
emg denoising method: rawdata
Signal Length: 14148
step3: Overlapping window segmentation...
       Overlapping window segmentation...
       emg_sample.shape:  (146, 128, 9) , movement_label.shape:  (146,)
step4: Feature extraction...
       emg_feas.shape:  (146, 9, 6)
step5: Merging: all emg data, features, labels...

******************************** motion: STDUP ********************************
step1: Load data and label...
step2: Data preprocessing...
emg denoising method: rawdata
Signal Length: 8760
step3: Overlapping window segmentation...
       Overlapping window segmentation...
       emg_sample.shape:  (89, 128, 9) , movement_label.shape:  (89,)
step4: Feature extraction...
       emg_feas.shape:  (89, 9, 6)
step5:

In [4]:
## Read the file, for check
file_path = os.path.join(os.getcwd(), 'SIAT_LLMD_trainData', emg_denoise_method)

for i in range(len(subjects_list_global)):
    subject = 'Sub'+subjects_list_global[i]
    printlog(info = ''.join(['Read file of: ', subject]), time = True, line_break = False)
    file_name = os.path.join(file_path,''.join([subject,'_targetTrainData.npz']))
    with open(file_name, 'rb') as f:
        sub_emg_sample = np.load(f)['sub_emg_sample']
        sub_emg_features = np.load(f)['sub_emg_features']
        sub_motion_label_raw = np.load(f)['sub_motion_label_raw']
        sub_motion_label_encoded = np.load(f)['sub_motion_label_encoded']
        sub_motion_label_onehot = np.load(f)['sub_motion_label_onehot']
    print('emg_sample.shape: ', sub_emg_sample.shape)
    print('emg_features.shape: ', sub_emg_features.shape)
    print('motion_label_raw.shape: ', sub_motion_label_raw.shape, ', movement_label_encoded.shape: ', sub_motion_label_encoded.shape, ', motion_label_onehot.shape: ', sub_motion_label_onehot.shape)


================================================================================2025-02-16 23:05:43
Read file of: Sub01
emg_sample.shape:  (690, 9, 128)
emg_features.shape:  (690, 9, 6)
motion_label_raw.shape:  (690,) , movement_label_encoded.shape:  (690,) , motion_label_onehot.shape:  (690, 5)

================================================================================2025-02-16 23:05:43
Read file of: Sub02
emg_sample.shape:  (483, 9, 128)
emg_features.shape:  (483, 9, 6)
motion_label_raw.shape:  (483,) , movement_label_encoded.shape:  (483,) , motion_label_onehot.shape:  (483, 5)

================================================================================2025-02-16 23:05:43
Read file of: Sub03
emg_sample.shape:  (523, 9, 128)
emg_features.shape:  (523, 9, 6)
motion_label_raw.shape:  (523,) , movement_label_encoded.shape:  (523,) , motion_label_onehot.shape:  (523, 5)

================================================================================2025-02-16 23:05:43
Read 